In [290]:
import math 
import numpy as np
from scipy.stats import norm
from datetime import date
import matplotlib.pyplot as plt
   
#store common attributes of fx option
'''
Make sure notional is in the LHS currency, as it's most commonly asked, e.g. AUD 100mio vs USD, EUR 35mio vs USD, USD 50mio vs JPY
%Foreign: default. This is the case for USDJPY, USDNJA etc.  You take the domestic amount (JPY), divide by spot and LHS notional
DomesticPips: this is the case in the brokers for AUDUSD, EURUSD.  LHS_notional x pips converts into domestic (USD) units
%Domestic: rare
Foreign_Pips: very rare
''' 

class FXOption(object):   #for NonDeliverable need to overwrite outright forward
    
    def __init__(self, CallPutFlag, S, K, sigma,  rFor, rDom, dateEnd, notional, premConvention):
        
        #date format: date(2017, 6, 30)
        #constructors 
        
        self.CallPutFlag = CallPutFlag
        self.S = S
        self.K = K
        self.sigma = sigma
        self.rFor = rFor
        self.rDom = rDom
        
        ###-------------------------------------
        # Convert date to unit of years
        ###--------------------------------
  
        self.dateEnd = dateEnd
        today = date.today()
        self.Tau = (self.dateEnd - today).days / 365.0
        
        if self.Tau == 0:
            raise ValueError('Option Expired')
            
        Tau = self.Tau
        #print('Time is :'+str(Tau)) #sanity check
        
        self.notional = notional
        self.premConvention = premConvention
        
        ###-------------------------------------
        # d1 and d2 from standard BlackScholes
        ###--------------------------------
        
        self.d1 = (math.log(S/ K) + (rDom - rFor + 0.5*pow(sigma,2))*Tau)/(sigma*math.sqrt(Tau))
        self.d2 = self.d1 - sigma * math.sqrt(Tau)
        
        ###--------------------------------
        #used in greeks calculations
        ###--------------------------------
        
        currency_basis = 0.0
        self.f =  self.S * math.exp( ((self.rDom - self.rFor) +currency_basis)*self.Tau) 
        #print('f: '+str(self.f))
        
        pi = 3.141592654
        #density function of d1
        self.nd_1 = (1 / pow(2 * pi,0.5)) * math.exp(-self.d1 * self.d1 / 2)
    
            
    def price(self):
        
        #rename constructor variables for readibility
        
        CallPutFlag, S, K, sigma, rFor, rDom, Tau, notional, premConvention = \
        self.CallPutFlag, self.S, self.K, self.sigma, self.rFor, self.rDom, self.Tau,\
        self.notional, self.premConvention

        if CallPutFlag == "Call":
    
            dummy = S * math.exp(-rFor * Tau) * norm.cdf(self.d1) - K * math.exp(-rDom * \
                                           Tau)* norm.cdf(self.d2)
            
        else:
            dummy = K * math.exp(-rDom * Tau) * norm.cdf(-self.d2) - S * math.exp(-rFor * \
                                          Tau)* norm.cdf(-self.d1)
        
        if premConvention == 'DomesticPips':
            premium = notional * dummy
        else:
            ## default setting:  USD10mio x 100yen/usd converted back to LHS ie USD
            premium = notional * (dummy / self.S)
            
        #return '{:,}'.format(premium)  #format method to have commas on long numbers
        return premium
    
    def delta(self, convention):
        
        '''
        SEF: spot delta Excluding premium in Foreign, this is the standard BS delta, tells how much foreign currency (AUD, EUR etc) needed
        'today to hedge, excluding premium
        SIF: includes premium, used when premium is in foreign units like USDJPY ..or AUDUSD with premium in AUD% instead of standard USD pips
        'like in the brokers
        SED: rare
        SID: rare
        FEF: normal blackscholes forward delta
        FIF: this is what we use for long-dated CrossYEN...BE CAREFUL with the forward as we got basis points now on top
        of usual InterestRates parity
        FED: rare
        FID: rare
        Notional should be in Foreign, LHS, currency, ie EUR...if not, convert by spot rate
        The extreme example of a deep in the money,  0.6000 AUD call / USD put: delta is 40ish% if premium is in AUD
        (as the buyer pays AUD 6mio or so, so the remaining risk is only on AUD 4mio) vs 100% delta if it were in USD. '''
        
        dummy = 0 
        if convention == 'SEF':
            if self.CallPutFlag == 'Call':
                dummy = math.exp(-self.rFor * self.Tau) * self.notional * self.CND(self.d1)
            else:
                dummy =math.exp(-self.rFor * self.Tau) * self.notional * self.CND(-self.d1)
                
        if convention == 'SIF':
            if self.CallPutFlag == 'Call':
                dummy = math.exp(-self.rDom * self.Tau) * self.notional * self.K * self.CND(self.d2)/ self.S
            else:
                 dummy = math.exp(-self.rDom * self.Tau) * self.notional * self.K * self.CND(-self.d2)/ self.S
                    
        if convention == 'FEF':
            
            if self.CallPutFlag == 'Call':
                dummy = self.notional * self.CND(self.d1)
            else:
                dummy = self.notional* self.CND(-self.d1)
                
        if convention == 'FIF':
            if self.CallPutFlag == 'Call':
            #be careful your forward is correct!
               dummy = self.notional * self.k * self.CND(self.d2) / self.f
            else: 
                dummy = self.notional * self.k * self.CND(-self.d2)/ self.f
        
        return '{:,}'.format(dummy)
    
    
    def gamma(self):
        #rename constructor variables for readibility
        CallPutFlag, S, K, sigma, rFor, rDom, Tau, notional, premConvention = \
        self.CallPutFlag, self.S, self.K, self.sigma, self.rFor, self.rDom,\
        self.Tau, self.notional, self.premConvention   
        
        b = self.rDom - self.rFor
        #gamma = notional* (math.exp(-rFor * Tau) / (S * sigma * pow(Tau, 0.5))) * self.nd_1
        gamma = notional* (math.exp( (-b + rFor) * Tau) / (S * sigma * pow(Tau, 0.5))) * self.nd_1
        return '{:,}'.format(gamma)
    
    def vega(self):
        b = self.rDom - self.rFor
        #vega = self.notional * (1/100) * (self.S * math.exp(-self.rFor * self.Tau) * pow(self.Tau, 0.5) * self.nd_1
        
        vega = self.notional * math.exp((b - self.rDom) * self.Tau) * self.nd_1 * pow(self.Tau, 0.5) / 100
        return '{:,}'.format(vega)
    
    #def vanna(self):
        
     #   vanna= -Exp(-riskfree_foreign * Tau) * nd_1 * (d2 / Sigma)
      #  return vanna
    
    #def volga(self):
        
     #   volga = S * Exp(-riskfree_foreign * Tau) * Sqr(Tau) * nd_1 * (d1 * d2 / Sigma)
      #  return volga
    
    def strike_from_delta(self, delta_percent, convention):
        
        '''For DeltaNeutralStraddle, we want delta_call + delta_put = 0, or delta_call = -delta_put
        ie, exp(-rf*Tau)*CND(d1) = --exp(-rf*Tau)*CND(-d1)---> CND(d1) = CND(-d1)---> d1 = -d1
        d1 = (log(F/K) + (sigma *sigma * 0.5* Tau ))/(sigma*sqrt(Tau) ) = 0
        log (F/K) = - 0.5 * sigma * sigma * Tau  --> K = F * exp(0.5 * sigma * sigma * Tau)...premium unadjusted like EURUSD
        K =  F* exp(-0.5 *sigma *sigma*Tau) ...premium adjusted like USDXYZ'''
        
        delta_target = delta_percent
        increment = 0.0001
        strike_guess = self.f #initial guess
        debugger = 0
        k_SEF = 0 #initialize 

        if delta_percent == "DNS":
            if convention == "SEF":
                dummy = self.f * math.exp(0.5 * self.sigma *self.sigma * self.Tau)  #premium unadjusted
            else:
                dummy = self * math.exp(-0.5 * self.sigma *self.sigma * self.Tau) #premium adjusted
            return dummy
    
       #Below finds strikes for non-straddles. If EXCLUDING premium, we can easily invert the Delta Function
   
        if convention == 'SEF':
            if self.CallPutFlag == 'Call':
                k_SEF = -self.sigma * pow(self.Tau, 0.5) * norm.ppf(delta_percent / math.exp(-self.rFor  * self.Tau)) + \
                0.5 * self.sigma * self.sigma * self.Tau
            else:
                delta_percent = -1.0 * delta_percent #ensure it's a put, delta is negative
                k_SEF = self.sigma * pow(self.Tau, 0.5) * norm.ppf(-delta_percent / math.exp(-self.rFor  * self.Tau)) + \
                0.5 * self.sigma * self.sigma * self.Tau
            dummy = self.f * math.exp(k_SEF)
            return dummy
        
        #If INCLUDING premium, we need a numerical procedure to find the strike. use below only for <1y, else need forward 
        if convention == 'SIF':
            if self.CallPutFlag == 'Call':
                
                delta_guess = 0.5
                
                while delta_guess > delta_target:  #vs do until delta_guess < delta_target
                    
                    d1_guess = (math.log(self.S/ strike_guess) + (self.rDom - self.rFor + 0.5*pow(self.sigma,2))*self.Tau) \
                    /(self.sigma*math.sqrt(self.Tau))
                    d2_guess = d1_guess - self.sigma * math.sqrt(self.Tau)
                    
                    delta_guess = math.exp(-self.rDom * self.Tau) * strike_guess * self.CND(d2_guess) /self.S
                
                    strike_guess += increment 
                    #print(strike_guess, delta_guess)
                    debugger += 1
                    
                         
            else: #for puts
                delta_guess = -0.5
                delta_target = -1 *delta_target
                while delta_guess < delta_target:
                    
                    d1_guess = (math.log(self.S/ strike_guess) + (self.rDom - self.rFor + 0.5*pow(self.sigma,2))*self.Tau) \
                    /(self.sigma*math.sqrt(self.Tau))
                    d2_guess = d1_guess - self.sigma * math.sqrt(self.Tau)
                    
                    delta_guess = math.exp(-self.rDom * self.Tau) * -1 *strike_guess * self.CND(-d2_guess)/ self.S
                    strike_guess -= increment
                    debugger += 1
            
            dummy = strike_guess
                    
            return dummy
                                 
    def CND(self, x):
        
        '''generally x will be d1 or d2
        CND is the cumulative normal distribution of a random variable, ie, the integral of the density function_
        from negative infinity up to the random variable. Describes probability will be found at a value <= to x.
        it's the NORMSDIST function in excel
        for negative values, like -1.96 it will be asymptotic to 0 (lower probabilities), at 0 it's 0.50 (fifty/fifty, _
        for large quantities (like > 1.96) asymptotic to 1 (certain probability).
        Graphed ti's the shape ofcall-spread. '''
    
        pi = 3.141592654
        l = abs(x)
        k = 1 / (1 + 0.2316419 * l)
        a1 = 0.31938153
        a2 = -0.356563782
        a3 = 1.781477937
        a4 = -1.821255978
        a5 = 1.330274429
        
        n = 1 / math.sqrt(2 * pi) * math.exp(-l * l / 2)
        
        CND = 1 - n * (a1 * k + \
                       a2 * pow(k, 2) + \
                       a3 * pow(k, 3) + \
                       a4 * pow(k, 4) + \
                       a5 * pow(k, 5))
        if x < 0:
            CND = 1 - CND
        return CND

    def graph(self, dS):
 
        'generate value given S'
        CallPutFlag, S, K, sigma, rFor, rDom, Tau, notional, premConvention = \
        self.CallPutFlag, self.S, self.K, self.sigma, self.rFor, self.rDom,\
        self.Tau, self.notional, self.premConvention
        
        #dS = 0.001
        path = []
        values = []
        #upper_bound = int(round(S/dS)+1)
        upper_bound = int(S * 1.15)
        for i in range(upper_bound):
            path.append(S)
            opt_price = FXOption(CallPutFlag, S, K, sigma,  rFor, rDom, self.dateEnd, notional,\
                                 premConvention).price()
            values.append(opt_price)
            S = S + dS

        #print(values)
        for s, payoff in zip(path, values):
            # print ('%.2f  %.4f' % (s, payoff_values)) #TypeError: a float is required
            print('%.2f  %.4f' %(s, payoff))
    

In [291]:
#(self, CallPutFlag, S, K, sigma,  rFor, rDom, dateEnd, notional, premConvention):
    
opt1 = FXOption('Put', 112.60, 113, 0.112,  0.01563, -0.00644, date(2018,10, 25), 10000000,"")
print(opt1.price())
print(opt1.delta('SIF'))
print(opt1.gamma())
print(opt1.vega())
print(opt1.strike_from_delta(0.15, 'SIF'))

557946.766164
6,145,719.917177373
336,076.19654983835
37,251.77806567061
98.97177336073075


In [292]:
#(self, CallPutFlag, S, K, sigma,  rFor, rDom, dateEnd, notional, premConvention):
    
opt1 = FXOption('Call', 64.838, 63.1929, 0.0889, 0.01549, 0.05526, date(2018,10, 22), 10000000,"")
print(opt1.price())
print(opt1.delta('SIF'))
print(opt1.gamma())
print(opt1.vega())
print(opt1.strike_from_delta(0.15, 'SIF'))

703827.011852
6,987,767.62193696
525,005.3798596165
27,915.277793639158
73.46646352128161


In [289]:
opt1.graph(0.03)

Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
Time is :0.915068493150685
T

In [26]:
opt.delta('')

-0.4897151492864029

In [13]:
 
opt.graph(0.1)


TypeError: unsupported operand type(s) for -: 'float' and 'datetime.date'

In [4]:
plt.plot(path, values, label = 'opt_values')
plt.xlabel('spot')
plt.ylabel('opt value')
plt.title('opt value vs spot')
plt.legend()
plt.show()

NameError: name 'path' is not defined

In [ ]:
from datetime import date

d0 = date(2017, 6, 30)
d1 = date(2017, 10, 3)
delta = d0 - d1
print(delta.days)
print(delta)

In [ ]:
FXOption('Call', 112.77, 110.88, 0.076, 0.009, -0.0048, 30/365, 10000000,'DomesticPips').price()

In [ ]:
#daily level coefficients
alpha2y = 1.356
beta2y = 0.92
x6m = 5.55
y2y = alpha2y + beta2y*x6m
print(y2y)

In [ ]:
#daily level coefficients
alpha3y = -0.009
beta3y = 1.087
x = 10.5
y3y = alpha3y + beta3y*x
print(y3y)

In [ ]:
#from a pivot , derive rest of curve
pivot = 10.5
#2y to 5yr
alphasKRW = [-0.009, 1.627, 2.477, 3.324]
betasKRW = [1.087, 0.983, 0.948, 0.903]
vols = []
#NO: for i in range(alphasKRW):
for i in range(len(alphasKRW)):
    print(i)


In [ ]:
#from a pivot , derive rest of curve
pivot = 10.5
#2y to 5yr
alphasKRW = [-0.06, 1.627, 2.477, 3.324]
betasKRW = [1.087, 0.983, 0.948, 0.903]
vols = []
for i in range(len(alphasKRW)):
    vols.append((alphasKRW[i] + pivot*betasKRW[i]))
    
print(vols)

In [ ]:
notional = 100000000
df = (1/1+0.012)
ING_receive = 0.0625
PV_receive  = notional*df*ING_receive

#rac payoutis:
ING_pay = 0.0725
PV_pay = -notional*df*ING_pay

NPV = PV_receive + PV_pay
print(NPV)

In [ ]:
print(notional*ING_pay)

In [ ]:
   def getStrikeFromDelta(self, delta, convention):
        #this is just a forward..be careful on long-dates that xxcy basis and 3m vs 6m stuff is added
        f = self.S * (1 + self.rDom * self.Tau) / (1 + self.rFor * self.Tau) 
        if self.Tau > 1.0:
            print('Check the forward')
            
'For DeltaNeutralStraddle, we want delta_call + delta_put = 0, or delta_call = -delta_put
'ie, exp(-rf*Tau)*CND(d1) = --exp(-rf*Tau)*CND(-d1)---> CND(d1) = CND(-d1)---> d1 = -d1
'd1 = (log(F/K) + (sigma *sigma * 0.5* Tau ))/(sigma*sqrt(Tau) ) = 0
'log (F/K) = - 0.5 * sigma * sigma * Tau  --> K = F * exp(0.5 * sigma * sigma * Tau)...premium unadjusted like EURUSD
' K =  F* exp(-0.5 *sigma *sigma*Tau) ...premium adjusted like USDXYZ
        if Delta = 'DNS':
            if DeltaConvention = "SEF":
                dummy = f * math.exp(0.5 * self.sigma * self.sigma * self.Tau)  'premium unadjusted
            elif:
                dummy = f * Exp(-0.5 * self.sigma * self.sigma * self.Tau) 'premium adjusted
        elif:


'Below finds strikes for non-straddles. If EXCLUDING premium, we can easily invert the Delta Function
'If INCLUDING premium, we need a numerical procedure to find the strike

    
Dim DeltaTarget As Double
Dim DeltaGuess As Double
Dim StrikeGuess As Double
Dim increment As Double
Dim Debugger As Integer

    DeltaTarget = Delta
    increment = 0.01
    StrikeGuess = f  'initial guess
    Debugger = 0
    
    Select Case DeltaConvention
     
        Case Is = "SEF"
            If CallPutFlag = "Call" Then
                x_SEF = -Sigma * Sqr(Tau) * Application.WorksheetFunction.NormSInv(Delta / Exp(-riskfree_foreign * Tau)) _
                + 0.5 * Sigma * Sigma * Tau
            Else
                Delta = -1 * Delta  'ensure that if it's a put, the delta is negative
                x_SEF = Sigma * Sqr(Tau) * Application.WorksheetFunction.NormSInv(-Delta / Exp(-riskfree_foreign * Tau)) _
                + 0.5 * Sigma * Sigma * Tau
            End If
            dummy = f * Exp(x_SEF)
            
        Case Is = "FEF"
            If CallPutFlag = "Call" Then
                x_SEF = -Sigma * Sqr(Tau) * Application.WorksheetFunction.NormSInv(Delta) _
                + 0.5 * Sigma * Sigma * Tau
            Else
                Delta = -1 * Delta  'ensure that if it's a put, the delta is negative
                x_SEF = Sigma * Sqr(Tau) * Application.WorksheetFunction.NormSInv(-Delta) _
                + 0.5 * Sigma * Sigma * Tau
            End If
            dummy = f * Exp(x_SEF)
            
'If INCLUDING premium, we need a numerical procedure to find the strike

        Case Is = "SIF"
            If CallPutFlag = "Call" Then
                DeltaGuess = 0.5 '* Notional
                    Do Until DeltaGuess < DeltaTarget
                        d1 = ((Log(S / StrikeGuess) + (riskfree_domestic - riskfree_foreign + Sigma * Sigma / 2) * Tau)) / (Sigma * Sqr(Tau))
                        d2 = d1 - Sigma * Sqr(Tau)
                        DeltaGuess = Exp(-riskfree_domestic * Tau) * StrikeGuess * CND(d2) / S
                        'exp(-rf*T)*CND(d1) - (OptionValue)/S = exp(-rf*T)*CND(d1) - (exp(-rf*T)*S*CND(d1)-K *exp(-rDom*T)*CND(d2))/S
                        '                                     = cancel out  + exp(-rDom*T) * K * CND(d2)/S
                        StrikeGuess = StrikeGuess + increment
                        Debugger = Debugger + 1
                        dummy = StrikeGuess
                    'If Debugger > 2000 Then Exit Do
                    Loop
            Else    'for puts
                DeltaGuess = (-0.5) ' * Notional)
                DeltaTarget = -1 * DeltaTarget
                    Do Until DeltaGuess > DeltaTarget
                        d1 = ((Log(S / StrikeGuess) + (riskfree_domestic - riskfree_foreign + Sigma * Sigma / 2) * Tau)) / (Sigma * Sqr(Tau))
                        d2 = d1 - Sigma * Sqr(Tau)
                        'DeltaGuess = Exp(-riskfree_domestic * Tau) * -Notional * StrikeGuess * CND(-d2) / S
                        DeltaGuess = Exp(-riskfree_domestic * Tau) * -1 * StrikeGuess * CND(-d2) / S 'notice the -1 else DeltaGuess turns positive
                        StrikeGuess = StrikeGuess - increment
                        Debugger = Debugger + 1
                        'If Debugger > 5000 Then Exit Do
                        dummy = StrikeGuess
                    Loop
            End If
    
        If Debugger > 5000 Then
            MsgBox (Debugger)
        End If
        
        Case Is = "FIF"
            If CallPutFlag = "Call" Then
                DeltaGuess = 0.5 '* Notional
                    Do Until DeltaGuess < DeltaTarget
                        d1 = ((Log(S / StrikeGuess) + (riskfree_domestic - riskfree_foreign + Sigma * Sigma / 2) * Tau)) / (Sigma * Sqr(Tau))
                        d2 = d1 - Sigma * Sqr(Tau)
                        DeltaGuess = StrikeGuess * CND(d2) / f
                        StrikeGuess = StrikeGuess + increment
                        Debugger = Debugger + 1
                        dummy = StrikeGuess
                    'If Debugger > 2000 Then Exit Do
                    Loop
            Else    'for puts
                DeltaGuess = (-0.5) ' * Notional)
                DeltaTarget = -1 * DeltaTarget
                    Do Until DeltaGuess > DeltaTarget
                        d1 = ((Log(S / StrikeGuess) + (riskfree_domestic - riskfree_foreign + Sigma * Sigma / 2) * Tau)) / (Sigma * Sqr(Tau))
                        d2 = d1 - Sigma * Sqr(Tau)
                        
                        DeltaGuess = -1 * StrikeGuess * CND(-d2) / f 'notice the -1 else DeltaGuess turns positive
                        StrikeGuess = StrikeGuess - increment
                        Debugger = Debugger + 1
                        'If Debugger > 5000 Then Exit Do
                        dummy = StrikeGuess
                    Loop
            End If
    
        If Debugger > 5000 Then
            MsgBox (Debugger)
        End If
    
    End Select
    
    TA_FXStrike_from_Delta = dummy    
    
   
           